<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml3/blob/main/term2_sprint18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# term2_sprint18.ipynb データセットの作成

## 1.このSprintについて

### Sprintの目的
- データセットを作成し学習する
- データ拡張を行う

# どのように学ぶか
自作のデータセットを作り、さらにデータ拡張も行なっていきます。

### 【問題1】自作データセットでの分類の学習
自作のデータセットに対して分類問題を解いてください。任意の実装を使用してください。

### 1.1.1（解答）前処理_学習用データの作成

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adagrad
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import random

In [ ]:
import os

os.chdir('/content/drive/My Drive/Colab Notebooks')

# カレントディレクトリの取得
print(os.getcwd())

In [ ]:
cd term2_sprint18_data/img

In [ ]:
rm -r .ipynb_checkpoints/

In [ ]:
cd cat

In [ ]:
rm -r .ipynb_checkpoints/

In [ ]:
cd ../../../

In [ ]:
# 学習用のデータを作る.
image_list = []
label_list = []

# ./data/train 以下のorange,appleディレクトリ以下の画像を読み込む。
for dir in os.listdir("term2_sprint18_data/img/"):
    if dir == ".DS_Store":
        continue

    dir1 = "term2_sprint18_data/img/" + dir 
    label = 0

    if dir == "cat":    # appleはラベル0
        label = 0
    elif dir == "dog": # orangeはラベル1
        label = 1

    for file in os.listdir(dir1):
        if file != ".DS_Store":
            # 配列label_listに正解ラベルを追加(cat:0 dog:1)
            label_list.append(label)
            filepath = dir1 + "/" + file
            # 画像を25x25pixelに変換し、1要素が[R,G,B]3要素を含む配列の25x25の２次元配列として読み込む。
            # [R,G,B]はそれぞれが0-255の配列。
            image = np.array(Image.open(filepath).resize((25, 25)))
            # print(filepath)
            # 配列を変換し、[[Redの配列],[Greenの配列],[Blueの配列]] のような形にする。
            image = image.transpose(2, 0, 1)
            # さらにフラットな1次元配列に変換。最初の1/3はRed、次がGreenの、最後がBlueの要素がフラットに並ぶ。
            image = image.reshape(1, image.shape[0] * image.shape[1] * image.shape[2]).astype("float32")[0]
            # 出来上がった配列をimage_listに追加。
            image_list.append(image / 255.)

# kerasに渡すためにnumpy配列に変換。
image_list = np.array(image_list)

# onehotエンコーディング
# cat = 0 -> [1,0], dog = 1 -> [0,1] 。
Y = to_categorical(label_list)

In [ ]:
image_list[0].shape

### 1.2.1（解答）kerasでのDeepLearning実装

In [ ]:
# モデルを生成してニューラルネットを構築
model = Sequential()
model.add(Dense(200, input_dim=1875))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(200))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(2))
model.add(Activation("softmax"))

# オプティマイザにAdamを使用
opt = Adam(lr=0.001)

# モデルをコンパイル
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# 学習を実行。10%はテストに使用。
model.fit(image_list, Y, epochs=50, batch_size=100, validation_split=0.1)

### 1.3.1（解答）予測

In [ ]:
from keras.preprocessing import image
import numpy as np
import sys

# 検証する画像データを読み込む
filepath = "term2_sprint18_data/test/cat2.jpg"
image = np.array(Image.open(filepath).resize((25, 25)))

# 画像データの型番の順番をmodelに合わせる
image = image.transpose(2, 0, 1)
# shape(1875,)の形に変換
image = image.reshape(1, image.shape[0] * image.shape[1] * image.shape[2]).astype("float32")[0]
result = model.predict(np.array([image / 255.]))
print("result:", result, "（ねこ : いぬ）")

## 【問題2】分類データセットに対するデータ拡張
データ拡張（Data Augmentation）を行ってください。


**《データ拡張のためのライブラリ》**


データ拡張にはalbumentationsなどがあります。


albu/albumentations: fast image augmentation library and easy to use wrapper around other libraries  
https://github.com/albu/albumentations


また、Kerasを使う場合はImageDataGeneratorも便利です。

### 2.1.1 読み込み画像の指定

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 読み込み画像へのpath
path      = "term2_sprint18_data/img/cat/" 
# 画像の名称
pic_name  = "0"
# 拡張子
extension = ".png"
# 生成された画像の格納path
save_path = "term2_sprint18_data/padding_img/cat/"
# 格納先ディレクトリが存在しなければ作成する
os.makedirs(save_path, exist_ok=True)

# 1-1. テスト用画像の読み込み
img = image.load_img(path + pic_name + extension)
# 1-2. 画像の配列化
img = np.array(img)

In [ ]:
# 2. 配列を四次元に整形
print("整形前の配列shape : {}".format(img.shape))
print("( 四次元配列でないとジェネレータに渡せないので、np.newaxisで次元追加 )")
img = img[np.newaxis]
print("追加後shape      : {}".format(img.shape))

### 2.2.1 ImageDataGeneratorの設定

In [ ]:
# 3. ジェネレータの定義
# データ拡張のパターンを指定する
glasses_gen = ImageDataGenerator(
    rotation_range=120, # 回転
    width_shift_range=0.2, # 左右スライド
    height_shift_range=0.2, # 上下スライド
    shear_range=40, # せん断写像
    zoom_range=0, # 拡大縮小
    horizontal_flip=True, # 左右反転
    vertical_flip=False) # 上下反転
 
 # 4. ジェネレータを生成。
gen = glasses_gen.flow(img, 
                     batch_size  = 1,         ## 起動バッチ数の指定。今回は画像1枚なので1
                     save_to_dir = "term2_sprint18_data/padding_img/cat", ## 保存先ディレクトリのpath
                     save_format = "jpg"      ## 保存形式の指定
                    )
  
plt.figure(figsize=(10,8))  # 表示画像サイズ

### 2.3.1 画像の表示

In [ ]:
# 5. 9枚の画像を生成する
for i in range(9):
 
    # ジェネレータにおけるイテレータを進める
    batches = next(gen)
    
    # 1次元目は次元合わせだったので読み飛ばす。
    # 画像として表示するために、floatからuint8に変換する。
    g_img = batches[0].astype(np.uint8)
 
    # matplotlib.pyplot.subplot
    #  ... 一つの図の中に複数のイメージを格納したいときに用いる。
    #      引数は前から、行数、列数、何番目のプロットとして表示するかの指定。
    #      下の記述で言えば、 3x3 の表示枠を一つの図中に用意し、順番に表示データを当て込んでいる。
    plt.subplot(3, 3, i+1)
    plt.imshow(g_img)
    
    # matplotlib.pyplot.axis
    # ... 座標軸を表示するかどうかを指定する。off 指定で表示されなくなる。
    plt.axis('off')
 
# 6. 表示
plt.show()

### 2.4.1 Albumentations

In [ ]:
!pip install -U albumentations

In [ ]:
import albumentations as A
import cv2


# Declare an augmentation pipeline
transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])

# Read an image with OpenCV and convert it to the RGB colorspace
image = cv2.imread("/content/drive/MyDrive/Colab Notebooks/term2_sprint18_data/img/cat/1.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Augment an image
transformed = transform(image=image)
transformed_image = transformed["image"]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(transformed_image)

## 【問題3】物体検出データセットの用意
次に、物体検出を行います。バウンディングボックスのアノテーションを行い物体検出のためのデータセットを作成してください。


《アノテーションのためのツール》


アノテーションツールにはLabelImgなどがあります。


tzutalin/labelImg: LabelImg is a graphical image annotation tool and label object bounding boxes in images
https://github.com/tzutalin/labelImg

### 3.1.1 （解答）Labellmgの初期設定

In [ ]:
!pip3 install pyqt5 lxml

In [ ]:
cd labelImg-master

In [ ]:
!make qt5py3

In [ ]:
!python3 labelImg.py

本来上記の手順でlabellmgが起動するはずがcolabでは起動できず、condaで起動。

In [ ]:
# 作成したデータ
labellmg_data = "/content/drive/MyDrive/Colab Notebooks/term2_sprint18_data/cat_xml/1.xml"

## 【問題4】物体検出データセットに対するデータ拡張
データ拡張（Data Augmentation）を行ってください。前述のalbumentationsはバウンディングボックスを合わせての加工が可能です。詳細はREADME.mdを確認してください。


物体検出の学習を行なうかどうかは任意とします。  

参考資料  
https://qiita.com/sino20023/items/0314438d397240e56576

### 4.1.1 （解答） XMLファイル操作

In [ ]:
pwd

In [ ]:
# 読み込み画像へのpath
path      = "term2_sprint18_data/cat_xml/" 
# 画像の名称
pic_name  = "1"
# 拡張子
extension = ".xml"
# 生成された画像の格納path
save_path = "term2_sprint18_data/padding_img/cat/"
# 格納先ディレクトリが存在しなければ作成する
os.makedirs(save_path, exist_ok=True)

In [ ]:
import xml.etree.ElementTree as ET
#xmlデータを読み込みます
img = ET.parse(path + pic_name + extension)
#一番上の階層の要素を取り出します
img = img.getroot()

In [ ]:
for i in img:
    print(i.tag)
    print(i.attrib)

In [ ]:
object = [] # 並び(xmin,ymin,xmax,ymax)
for name in img.iter('object'):
    for i in name:
        object.append(i.text)
object

In [ ]:
bndbox = [] # 並び(xmin,ymin,xmax,ymax)
for name in img.iter('bndbox'):
    for i in name:
        bndbox.append(i.text)
bndbox

In [ ]:
size = [] # 並び(width,height,depth)
for name in img.iter('size'):
    for i in name:
        size.append(i.text)
size

### 4.2.1 （解答） データ拡張

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/term2_sprint18_data/img/cat/1.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
# x_min、y_min、width、height]
bboxes = [bndbox]
category_ids = [object[3]]

# We will use the mapping from category_id to the class name
# to visualize the class label for the bounding box on the image
category_id_to_name = {object[3]: object[0]}

In [ ]:
visualize(image, bboxes, category_ids, category_id_to_name)